In [41]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd

from PIL import Image
from fastai.learner import load_learner
from torchvision.transforms import ToTensor
from responsibleai_vision.common.constants import ImageColumns
from tqdm import tqdm

BATCH_SIZE = 4
DATASET_NAME = "CIFAR"

dataset = torchvision.datasets.CIFAR10(root="data", train=False, download=True, transform=ToTensor())
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

data = pd.DataFrame(columns=[ImageColumns.IMAGE.value,
                             ImageColumns.LABEL.value])

cnts = {}
data_path = "data/" + DATASET_NAME + "/" 

for batch in tqdm(dataloader):
    images, labels = batch
    for i in range(BATCH_SIZE):
        img, label = images[i], labels[i]
        label = str(label.numpy())
        img = img.numpy()
        img = np.transpose(img, (1, 2, 0))
        img = Image.fromarray((img * 255).astype(np.uint8))
        
        if label not in cnts:
            os.makedirs(data_path + label, exist_ok=True)
            cnts[label] = 1
        else:
            cnts[label] += 1
        
        img_path = data_path + label + "/class_" + label + "_img_" + str(cnts[label]) + ".jpeg"
        img.save(img_path)
        item = pd.DataFrame(data={ImageColumns.IMAGE.value: [img_path], 
                                  ImageColumns.LABEL.value: [label]})
        data = pd.concat([data, item], ignore_index=True)
        

Files already downloaded and verified


100%|██████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:27<00:00, 91.45it/s]
